In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import zipfile
import rarfile
import py7zr
import magic
import re
import shutil
import io
import pyarrow as pa
import pyarrow.parquet as pq
import glob

In [ ]:
url = 'https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enade'
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
content_core = soup.find(id="content-core")
links = content_core.find_all("a")
hrefs = [link.get("href") for link in links if link.get("href")]


###### Stream=True divide os dados(chunk) para aumentar a eficiência e auxiliar a memória
###### iter_content é usado para binários(videos,imagens,zip)
###### iter_lines é usaod para arquivos de texto(csv,log)


In [ ]:
def setup_folders():
    download_folder = "downloads"
    extract_folder = "extracted_files"
    transformed_parquet_files = "transformed_parquet_files"
    os.makedirs(download_folder, exist_ok=True)
    os.makedirs(extract_folder, exist_ok=True)
    os.makedirs(transformed_parquet_files, exist_ok=True)
    return download_folder, extract_folder, transformed_parquet_files

download_folder, extract_folder, transformed_parquet_files = setup_folders()


In [ ]:
def baixar(url):    
    
    try:        
        filename = os.path.join(download_folder, os.path.basename(url))  
        response = requests.get(url, stream=True, timeout=30) 

        if response.status_code == 200:  #code status
            with open(filename, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024): 
                    f.write(chunk)
            print(f"Download concluído: {filename}")
        else:
            print(f"Erro ao baixar {url} (Status Code: {response.status_code})")

    except requests.exceptions.RequestException as e: 
        print(f" Erro ao baixar {url}: {e}")

for href in hrefs:  
    baixar(href)  

In [ ]:
def detectar_tipo_arquivo(filename):
    tipo = magic.Magic(mime=True).from_file(filename)
    print(filename)

    if "zip" in tipo:
        return "zip"
    elif "rar" in tipo:
        return "rar"
    elif "7z" in tipo or "x-7z-compressed" in tipo:
        return "7z"
    else:
        return None

In [ ]:
def extrair(filename, extract_folder):
    try:
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Arquivo {filename} não encontrado.")
        
        formato_real = detectar_tipo_arquivo(filename)
        
        if formato_real == "zip":
            with zipfile.ZipFile(filename, "r") as zip_ref:
                zip_ref.extractall(extract_folder)
            print(f"Extraído para: {extract_folder}")
        
        elif formato_real == "rar":
            with rarfile.RarFile(filename) as rar_ref:
                rar_ref.extractall(extract_folder)
        
        elif formato_real == "7z":
            with py7zr.SevenZipFile(filename, "r") as sevenz_ref:
                sevenz_ref.extractall(extract_folder)

    except zipfile.BadZipFile as e:
        print(f"Erro: {e}")
    except rarfile.BadRarFile:
        print(f"Erro: O arquivo RAR {filename} está corrompido ou inválido.")
    except py7zr.Bad7zFile:
        print(f"Erro: O arquivo 7Z {filename} está corrompido ou inválido.")
    except PermissionError:
        print(f"Erro: Permissão negada para extrair arquivos em {extract_folder}.")
    except Exception as e:
        print(f"Erro inesperado ao extrair {filename}: {e}")
            

In [ ]:
for href in hrefs: 
    filename = os.path.join(download_folder, os.path.basename(href)) 
    extrair(filename, extract_folder) #só o rar que não funcionou

### criando estrutura de pastas do transformed_parquet_files

In [ ]:
root_dir = "/home/951551482@adm.unifor.br/Documentos/microdados-enade/extracted_files"
base_destination = "/home/951551482@adm.unifor.br/Documentos/microdados-enade/transformed_parquet_files"

for year in range(2013, 2022):

    possible_folders = [
        os.path.join(root_dir, f"microdados_Enade_{year}_LGPD", "2.DADOS"),
        os.path.join(root_dir, f"microdados_Enade_{year}_LGPD", "2. DADOS")
    ]

    folder_path = next((folder for folder in possible_folders if os.path.exists(folder)), None)
    if folder_path: #adm.unifor.br/Documentos/microdados-enade/extracted_files/microdados_Enade_2013_LGPD/2.DADOS
        destination_folder = os.path.join(base_destination, f"microdados_Enade_{year}_LGPD") #@adm.unifor.br/Documentos/microdados-enade/transformed_parquet_files/microdados_Enade_2013_LGPD
        os.makedirs(destination_folder, exist_ok=True)  # Garante que o diretório de destino exista
        for file in os.listdir(folder_path):
            if file.startswith(f"microdados{year}_arq") and file.endswith(".txt"): #microdados2013_arq1.txt
                source_file = os.path.join(folder_path, file) #2@adm.unifor.br/Documentos/microdados-enade/extracted_files/microdados_Enade_2013_LGPD/2.DADOS/microdados2013_arq1.txt
                parquet_file = os.path.join(destination_folder, f"{os.path.splitext(file)[0]}.parquet") #nome do arquivo parquet
                #df
                try:
                    df = pd.read_csv(source_file, delimiter=';', dtype=str , low_memory=False)
                except Exception as e:
                    print(f"Erro ao ler o arquivo {source_file}: {e}")
                    continue

                table = pa.Table.from_pandas(df)

                try:
                    pq.write_table(table, parquet_file)
                    print(f'Arquivo Parquet criado: {parquet_file}')
                except Exception as e:
                    print(f"Erro ao escrever o arquivo Parquet {parquet_file}: {e}")
                    continue
            
            
               
       

                   
               

            
                    
                                        

In [31]:
base_dir = "/home/951551482@adm.unifor.br/Documentos/microdados-enade/transformed_parquet_files"
dataframes = {}

for root, dirs, files in os.walk(base_dir): #gera o nome das coisas em uma árvore de diretórios
    for file in files:
        if file.endswith(".parquet"):
            sufixo = file.split('_')[-1].split('.')[0] #arq1 ....arq43
            file_path = os.path.join(root, file)
            df = pd.read_parquet(file_path)

            

# for sufixo, dfs in dataframes.items():
#     print(sufixo)
    # combined_df = pd.concat(dfs, ignore_index=True, axis=0)
    # print(f"DataFrame combinado para {sufixo}:")
    # print(combined_df)
    # print("\n")


In [1]:
import os
import pandas as pd

base_dir = "/home/951551482@adm.unifor.br/Documentos/microdados-enade/transformed_parquet_files"
dataframes = {}

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".parquet"):
            sufixo = file.split('_')[-1].split('.')[0]  # Extrai o sufixo numérico (arq1, arq2, ..., arq43)
            file_path = os.path.join(root, file)
            df = pd.read_parquet(file_path)
            
            # Verifica se o sufixo já existe no dicionário
            if sufixo in dataframes:
                # Se existir, concatena o DataFrame atual com o existente
                dataframes[sufixo] = pd.concat([dataframes[sufixo], df], ignore_index=True)
            else:
                # Se não existir, cria uma nova entrada no dicionário
                dataframes[sufixo] = df

# Agora, dataframes contém os DataFrames agrupados por sufixo
# Por exemplo, dataframes['arq1'] contém todos os DataFrames concatenados com sufixo 'arq1'

In [33]:
dataframes['arq10']

,NU_ANO,CO_CURSO,QE_I04
0,2013,2214,A
1,2013,2214,A
2,2013,11313,A
3,2013,11313,A
4,2013,11313,A
...,...,...,...
3452045,2021,70985,None
3452046,2021,70985,None
3452047,2021,70985,None
3452048,2021,70985,None
